## Test Markdown for Task 1

We present a novel dataset based on Sentinel-2 satellite images covering <span style="color:blue; font-weight:bold;">13 spectral bands</span> and consisting out of <span style="color:blue; font-weight:bold;">10 classes</span> with in total <span style="color:blue; font-weight:bold;">27,000 labeled and geo-referenced images</span>. We provide benchmarks for this novel dataset with its spectral bands using state-of-the-art deep Convolutional Neural Network (CNNs). With the proposed novel dataset, we achieved an overall classification accuracy of <span style="color:red; font-weight:bold;">98.57%</span>. The resulting classification system opens a gate towards a number of Earth observation applications. We demonstrate how this classification system can be used for detecting land use and land cover changes and how it can assist in improving geographical maps. The geo-referenced dataset EuroSAT is made publicly available here.

Things to understand:
    1. Spectral bands?
    2. 10 classes?
    3. Labeled and geo-referenced images

In [15]:
import numpy as np
file_path = "Datasets/x_train.npy"
data = np.load(file_path)
print(data.shape)
print(data.dtype.names)
print(data[:5])

(16200, 512)
